In [107]:
import numpy as np
import pandas as pd


In [108]:
#from google.colab import drive

#drive.mount('/content/drive')

file_path = 'faithful.dat'


In [109]:
data = pd.read_table(file_path, sep="\s+", index_col=0)
data.info

<bound method DataFrame.info of      eruptions  waiting
1        3.600       79
2        1.800       54
3        3.333       74
4        2.283       62
5        4.533       85
..         ...      ...
268      4.117       81
269      2.150       46
270      4.417       90
271      1.817       46
272      4.467       74

[272 rows x 2 columns]>

In [110]:
import numpy as np
import pdb

def E_step(X, means, covariance, prob):

    """
    Compute the responsibilities of each data point for each component.

    Parameters:
    - X: Input data (n x d)
    - means: Matrix of means for each component (d x G).
    - covariance: Shared covariance matrix (d x d).
    - prob: A G-dimensional probability vector (p1,…,pG)

    Returns:
    - gamma: An n x G matrix where each (i, j) entry represents P(Zi=k | xi).
    """
    n, d = X.shape
    G = len(prob)

    gamma = np.zeros((n, G))


    for k in range(G):
        mean_k = means[:, k]
        prob_k = prob[k]

        datanp = data.to_numpy()[:, np.newaxis] # add a new first dimension to x
        diff = datanp - mean_k.T
        inv_cov = np.linalg.inv(covariance)

        prefactor = (2*np.pi)**-(d/2)*np.linalg.det(inv_cov)
        exp_term = np.exp(-np.einsum('ijk, kl, ijl->ij', diff, inv_cov, diff) / 2)

        gamma_k = prefactor * exp_term
        gamma_k = gamma_k * prob_k

        gamma[:, k] = gamma_k.ravel()

    # Normalize the responsibilities so that they sum to 1 for each data point
    gamma = gamma / np.sum(gamma, axis=1, keepdims=True)

    return gamma



In [111]:
def M_step(X, gamma):

    n, d = X.shape
    n, G = gamma.shape

    # Update mean
    gamma_sum = np.sum(gamma,axis=0) #1 x G
    updated_mean = np.dot(gamma.T, data)/gamma_sum.T[:, np.newaxis] #G x d

    updated_mean_t = updated_mean.T #d x G

    # Update probs for component k
    updated_probs = gamma_sum/n

    # Update covariance for component k
    shared_covariance = np.zeros((d, d))

    for k in range(G):
        diff_k = data - updated_mean[k][np.newaxis,:]
        weighted_covariance = np.dot(diff_k.T, (gamma[:, k][:, np.newaxis] * diff_k))

        shared_covariance += weighted_covariance

    total_responsibilities = np.sum(gamma, axis=0)
    shared_covariance /= np.sum(total_responsibilities)

    return updated_mean_t, shared_covariance, updated_probs

In [112]:
def log_likelihood(X, means, covariance, prob):
    """
    Compute the log-likelihood of the data given the parameters of GMM

    Parameters:
    - X: Input data (n x d)
    - means: Matrix of means for each component (d x G).
    - covariance: Shared covariance matrix (d x d).
    - prob: A G-dimensional probability vector (p1,…,pG)

    Returns:
    - log_lik: The log-likelihood of the data given the model parameters.
    """
    n, d = X.shape
    G = len(prob)

    likelihood = np.zeros((n, G))


    for k in range(G):
        mean_k = means[:, k]
        prob_k = prob[k]

        datanp = data.to_numpy()[:, np.newaxis] # add a new first dimension to x
        diff = datanp - mean_k.T
        inv_cov = np.linalg.inv(covariance)

        prefactor = 1/np.sqrt(((2*np.pi)**(d))* np.linalg.det(covariance))
        exp_term = np.exp(-np.einsum('ijk, kl, ijl->ij', diff, inv_cov, diff) / 2)

        pdf_k = prefactor * exp_term
        multi_pdf_k = pdf_k * prob_k

        likelihood[:, k] = multi_pdf_k.ravel()

    #print(f"log_likelihood:likelihood={likelihood}")
    likelihood_G = np.sum(likelihood, axis=1, keepdims=True)

    log_lik = np.sum(np.log(likelihood_G))

    return log_lik


In [113]:
def test_log_likelihood(X, means, covariances, mixing_coefficients):
    """
    Compute the log-likelihood of the data given the parameters of a Gaussian Mixture Model (GMM).

    Parameters:
    - X: Input data (n x d), where n is the number of data points and d is the dimensionality.
    - means: List of means for each component (G x d).
    - covariances: List of covariance matrices for each component (G x d x d).
    - mixing_coefficients: List of mixing coefficients for each component (G).

    Returns:
    - log_lik: The log-likelihood of the data given the model parameters.
    """
    n, d = X.shape
    G = len(means)

    log_lik = 0

    for i in range(n):
        data_point = X.to_numpy()[i, :]
        likelihood_i = 0

        for k in range(G):
            mean_k = means[k]
            covariance_k = covariances[k]
            mixing_coefficient_k = mixing_coefficients[k]

            # Calculate the likelihood of the data point for component k
            diff = data_point - mean_k
            exponent = -0.5 * np.dot(np.dot(diff, np.linalg.inv(covariance_k)), diff)
            likelihood_k = mixing_coefficient_k * (1.0 / np.sqrt((2 * np.pi) ** d * np.linalg.det(covariance_k))) * np.exp(exponent)
            likelihood_i += likelihood_k

        log_lik += np.log(likelihood_i)

    return log_lik


In [114]:
def myEM(data, G,means, covariance, prob):

    n, p = data.shape

    update_means = means
    update_covariance = covariance
    update_prob = prob


    for iteration in range(20):

        # E-step: Calculate gamma
        gamma = E_step(data, update_means, update_covariance, update_prob)
        #print(f"gamma={gamma}")

        # M-step: Update parameters
        M_means, M_covariance, M_prob = M_step(data, gamma)

        update_means = M_means
        update_covariance = M_covariance
        update_prob = M_prob

        print ("update_prob: ",update_prob)
        print ("update_means: ",update_means)
        print ("update_covariance: ",update_covariance)

    # Calculate the log-likelihood
    loglik = log_likelihood(data, update_means, update_covariance, update_prob)
    #loglik = loglik_fn(data, G, update_prob, update_means, update_covariance)

    return update_means, update_covariance, update_prob,loglik


In [115]:
def testing(data,G):

    n, p = data.shape

    if G==2:
      # Set initial values
      p1 = 10 / n
      p2 = 1 - p1
      initial_means = np.array([data[:10].mean(axis=0), data[10:].mean(axis=0)]).T
      diff1 = data[:10] - initial_means[:, 0]
      diff2 = data[10:] - initial_means[:, 1]
      initial_covariance = (1 / n) * (np.dot(diff1.T, diff1) + np.dot(diff2.T, diff2))

      initial_mixing_coefficients = np.array([p1, p2])

    if G == 3:
      # Set initial values
      p1 = 10 / n
      p2 = 20 / n
      p3 = 1 - p1 - p2
      initial_means = np.array([data[:10].mean(axis=0), data[10:30].mean(axis=0), data[30:].mean(axis=0)]).T
      print (initial_means)

      diff1 = data[:10] - initial_means[:, 0]
      diff2 = data[10:30] - initial_means[:, 1]
      diff3 = data[30:] - initial_means[:, 2]

      initial_covariance = (1 / n) * (np.dot(diff1.T, diff1) + np.dot(diff2.T, diff2) + np.dot(diff3.T, diff3))

      initial_mixing_coefficients = np.array([p1, p2, p3])


    # Perform the EM algorithm
    print(f"data={data}")
    print(f"G={G}")
    print(f"initial_means={initial_means}")
    print(f"initial_covariance={initial_covariance}")
    print(f"initial_mixing_coefficients={initial_mixing_coefficients}")
    mean, covariance, probs,loglik = myEM(data,G, initial_means, initial_covariance, initial_mixing_coefficients)

    print ("prob: ",probs)
    print ("mean: ",mean)
    print ("Sigma: ",covariance)
    print ("loglik: ", loglik)



In [116]:
testing(data,G=2)

#[6.80495600e-04 1.11838940e-02]
#[3.48138042e-04 7.38865927e-03]


data=     eruptions  waiting
1        3.600       79
2        1.800       54
3        3.333       74
4        2.283       62
5        4.533       85
..         ...      ...
268      4.117       81
269      2.150       46
270      4.417       90
271      1.817       46
272      4.467       74

[272 rows x 2 columns]
G=2
initial_means=[[ 3.3032      3.49482824]
 [71.8        70.86259542]]
initial_covariance=[[  1.29663847  13.93278021]
 [ 13.93278021 184.11269645]]
initial_mixing_coefficients=[0.03676471 0.96323529]
update_prob:  [0.03685524 0.96314476]
update_means:  [[ 3.31211483  3.49450513]
 [71.98706055 70.85534934]]
update_covariance:  [[  1.29675804  13.93374588]
 [ 13.93374588 184.09835147]]
update_prob:  [0.03695665 0.96304335]
update_means:  [[ 3.32141524  3.49416743]
 [72.18428601 70.84766166]]
update_covariance:  [[  1.29687674  13.93463695]
 [ 13.93463695 184.08022951]]
update_prob:  [0.03707039 0.96292961]
update_means:  [[ 3.33107607  3.49381592]
 [72.39169037 70.83951923]

In [117]:
y = np.array([[6.80495600e-04, 1.11838940e-02], [3.48138042e-04, 7.38865927e-03]])
q = np.array([[1.01917039e-04, 1.67499886e-03], [5.21402319e-05, 1.10659095e-03]])

y/q

array([[6.67695615, 6.67695619],
       [6.67695615, 6.67695617]])

In [118]:
testing(data,G=3)

[[ 3.3032      3.175       3.52126033]
 [71.8        68.25       71.0785124 ]]
data=     eruptions  waiting
1        3.600       79
2        1.800       54
3        3.333       74
4        2.283       62
5        4.533       85
..         ...      ...
268      4.117       81
269      2.150       46
270      4.417       90
271      1.817       46
272      4.467       74

[272 rows x 2 columns]
G=3
initial_means=[[ 3.3032      3.175       3.52126033]
 [71.8        68.25       71.0785124 ]]
initial_covariance=[[  1.28849554  13.8662627 ]
 [ 13.8662627  183.56933185]]
initial_mixing_coefficients=[0.03676471 0.07352941 0.88970588]
update_prob:  [0.03685998 0.07356745 0.88957258]
update_means:  [[ 3.31195271  3.16652085  3.52163704]
 [71.98913031 68.1379501  71.07998583]]
update_covariance:  [[  1.28818693  13.86277773]
 [ 13.86277773 183.5100423 ]]
update_prob:  [0.03696697 0.07360941 0.88942361]
update_means:  [[ 3.32116885  3.15747042  3.52204499]
 [72.18966834 68.01771887 71.08163078]]
u